In [1]:
import numpy as np
import pandas as pd
import math

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.max_rows = None  # 모든 row 내용 print
pd.options.display.max_rows = 20

In [3]:
# Provide the names for the columns since the CSV file with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

In [4]:
# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                     sep=',', names=feature_names, header=None, encoding='latin-1')

In [5]:
car_data = car_data.reindex(np.random.permutation(car_data.index))
# or equivalently,
# car_data = car_data.sample(frac=1)

In [6]:
print("Data set loaded. Num examples: ", len(car_data))

Data set loaded. Num examples:  205


In [7]:
car_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
182,2,122,volkswagen,diesel,std,two,sedan,fwd,front,97.30,...,97,idi,3.01,3.40,23.00,52,4800,37,46,7775
157,0,91,toyota,gas,std,four,hatchback,fwd,front,95.70,...,98,2bbl,3.19,3.03,9.00,70,4800,30,37,7198
55,3,150,mazda,gas,std,two,hatchback,rwd,front,95.30,...,70,4bbl,?,?,9.40,101,6000,17,23,10945
87,1,125,mitsubishi,gas,turbo,four,sedan,fwd,front,96.30,...,110,spdi,3.17,3.46,7.50,116,5500,23,30,9279
82,3,?,mitsubishi,gas,turbo,two,hatchback,fwd,front,95.90,...,156,spdi,3.58,3.86,7.00,145,5000,19,24,12629


In [8]:
LABEL = 'price'

numeric_feature_names = ['symboling', 'normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio']

In [9]:
# Run to inspect numeric features.
car_data[numeric_feature_names]

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio
182,2,122,97.30,171.70,65.50,55.70,2261,97,52,4800,37,46,3.01,3.40,23.00
157,0,91,95.70,166.30,64.40,52.80,2109,98,70,4800,30,37,3.19,3.03,9.00
55,3,150,95.30,169.00,65.70,49.60,2380,70,101,6000,17,23,?,?,9.40
87,1,125,96.30,172.40,65.40,51.60,2403,110,116,5500,23,30,3.17,3.46,7.50
82,3,?,95.90,173.20,66.30,50.20,2833,156,145,5000,19,24,3.58,3.86,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,1,168,94.50,168.70,64.00,52.60,2300,98,112,6600,26,29,3.24,3.08,9.40
94,1,128,94.50,165.30,63.80,54.50,1951,97,69,5200,31,37,3.15,3.29,9.40
1,3,?,88.60,168.80,64.10,48.80,2548,130,111,5000,21,27,3.47,2.68,9.00
195,-1,74,104.30,188.80,67.20,57.50,3034,141,114,5400,23,28,3.78,3.15,9.50


In [10]:
car_data[numeric_feature_names].describe()

,symboling,wheel-base,length,width,height,weight,engine-size,city-mpg,highway-mpg,compression-ratio
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,25.22,30.75,10.14
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,6.54,6.89,3.97
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,13.00,16.00,7.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,19.00,25.00,8.60
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,24.00,30.00,9.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,30.00,34.00,9.40
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,49.00,54.00,23.00


In [11]:
# Coerce the numeric features to numbers. This is necessary because the model crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
    car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors ='coerce')

"""
errors: error는 총 3개의 옵션이 존재합니다.
- errors = 'ignore' -> 만약 숫자로 변경할 수 없는 데이터라면 숫자로 변경하지 않고 원본 데이터를 그대로 반환합니다.
- errors = 'coerce' -> 만약 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 설정하여 반환합니다.
- errors = 'raise' -> 만약 숫자로 변경할 수 없는 데이터라면 에러를 일으키며 코드를 중단합니다."""
    
# Fill missing values with 0.
# Is this an OK thing to do? You may want to come back and revisit this decision later.
car_data.fillna(0, inplace=True)

In [12]:
car_data.shape

(205, 26)

In [13]:
X = car_data[numeric_feature_names].copy()
y = car_data['price'].copy()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

### normalization

In [15]:
car_data_norm = pd.DataFrame()
for feature_name in numeric_feature_names + [LABEL]:
    car_data_norm[feature_name] = (car_data[feature_name]-car_data[feature_name].mean())/car_data[feature_name].std()

In [16]:
car_data_norm.head()

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio,price
182,0.94,0.42,-0.24,-0.19,-0.19,0.81,-0.57,-0.72,-1.25,-0.40,1.80,2.21,-0.48,0.38,3.24,-0.64
157,-0.67,-0.11,-0.51,-0.63,-0.70,-0.38,-0.86,-0.69,-0.81,-0.40,0.73,0.91,-0.14,-0.29,-0.29,-0.71
55,1.74,0.90,-0.57,-0.41,-0.10,-1.69,-0.34,-1.37,-0.05,1.33,-1.26,-1.13,-6.10,-5.81,-0.19,-0.25
87,0.13,0.47,-0.41,-0.13,-0.24,-0.87,-0.29,-0.41,0.31,0.61,-0.34,-0.11,-0.18,0.49,-0.67,-0.45
82,1.74,-1.67,-0.47,-0.07,0.18,-1.44,0.53,0.70,1.02,-0.11,-0.95,-0.98,0.59,1.22,-0.79,-0.04


In [17]:
X_norm = car_data_norm.copy()
y_norm = X_norm.pop('price')
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_size=0.2)

In [18]:
X_train_norm_dict = {name:np.array(value) for name, value in X_train_norm.items()}
y_train_norm_dict = {y_train_norm.name:y_train_norm.values}

In [19]:
feature_columns = []
feature_columns = [
    tf.feature_column.numeric_column(feature_name)
    for feature_name in ['symboling', 'width']
]

In [20]:
len(feature_columns)

2

In [21]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [22]:
X_train_dict = {name:np.array(value) for name, value in X_train.items()}

In [23]:
print(feature_layer(X_train_norm_dict))

tf.Tensor(
[[-0.669832   -0.23671637]
 [ 0.13318297 -0.7961038 ]
 [-1.472847    0.2760554 ]
 [ 0.13318297  2.5602207 ]
 [ 1.7392129  -0.42317885]
 [-1.472847    2.7000675 ]
 [-1.472847    2.0474489 ]
 [ 0.13318297 -0.98256624]
 [ 0.13318297  0.4625179 ]
 [ 0.13318297 -0.98256624]
 [ 0.93619794 -0.19010076]
 [ 1.7392129  -0.8427194 ]
 [ 0.93619794 -0.93595064]
 [ 0.13318297 -1.0757974 ]
 [-0.669832    0.2760554 ]
 [-0.669832   -0.32994762]
 [ 1.7392129   0.2760554 ]
 [-0.669832   -0.93595064]
 [ 1.7392129   0.18282418]
 [ 0.93619794  0.2294398 ]
 [ 1.7392129   1.1151365 ]
 [ 0.13318297 -0.98256624]
 [ 0.13318297  0.2760554 ]
 [-0.669832    0.2760554 ]
 [-1.472847   -0.6096413 ]
 [-0.669832   -0.7028725 ]
 [-0.669832    2.0474489 ]
 [-1.472847    0.2760554 ]
 [-0.669832    0.2760554 ]
 [ 0.13318297 -0.7961038 ]
 [-2.275862    0.6023647 ]
 [ 0.13318297 -0.98256624]
 [ 0.93619794  0.18282418]
 [ 0.93619794 -0.19010076]
 [ 0.13318297 -0.98256624]
 [ 1.7392129   0.2760554 ]
 [ 1.7392129  -0.

In [24]:
feature_columns = []
feature_columns = [
    tf.feature_column.numeric_column(feature_name)
    for feature_name in numeric_feature_names
]
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

### convert dataframe to dict
when using feature_layer, <span style='color:red'> use dict </span> for input data in model.fit

In [25]:
X_train_norm_dict = {name:np.array(value) for name, value in X_train_norm.items()}
y_train_norm_dict = {y_train_norm.name:y_train_norm.values}

In [26]:
print(feature_layer(X_train_norm_dict))

tf.Tensor(
[[ 0.66362983 -0.18640871 -0.28764477 ... -0.32758275 -0.29170555
  -0.23671637]
 [-0.43862268  0.8835773  -0.28764477 ... -1.1726311  -0.939355
  -0.7961038 ]
 [ 0.00975122 -0.9506844  -0.23729281 ...  1.1435698   0.9537743
   0.2760554 ]
 ...
 [ 0.51417184 -0.6449741  -0.21211682 ...  0.38878787  0.0570288
   0.2760554 ]
 [-0.2518002  -1.4092499  -0.79116434 ...  0.95535445  0.12345439
   0.92867404]
 [ 0.66362983 -1.2563947  -0.66528445 ...  1.1550932   0.9205615
   0.6023647 ]], shape=(164, 15), dtype=float32)


In [27]:
model = tf.keras.models.Sequential()
model.add(feature_layer)
model.add(tf.keras.layers.Dense(units=3, input_shape=(1,), activation='relu'))
# model.add(tf.keras.layers.Dense(units=1, activation='relu')) 
#                         feature_layer일 경우 input_dim은 무조건 1:  바로 앞의 layer 수이므로 항상 1
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [28]:
# X_train_norm_dict = {name:np.array(value) for name, value in car_data_norm.items()}
# # Caution: dict 만들때 주의
# y_train_norm_dict = np.array(X_train_norm_dict.pop('price'))

###  feature column사용시 <span style='color:red'> .fit 입력 주의 </span>
 feature column 사용시는 <span style='color:red'>  y value는 array만 사용</span>. x value는 dict 사용

In [29]:
batch_size=16
epochs=500
model.fit(X_train_norm_dict, y_train_norm_dict.values(), epochs=epochs, batch_size= batch_size)
# feature column 사용시는 y value는 array만 사용. x value는 dict 사용

Epoch 1/500
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
11/11 [==============================] - 0s 701us/step - loss: 1.2022 - mean_squared_error: 1.2022
Epoch 2/500
11/11 [==============================] - 0s 698us/step - loss: 0.7071 - mean_squared_error: 0.7071
Epoch 3/500
11/11 [==============================] - 0s 665us/step - loss: 0.5581 - mean_squared_error: 0.5581
Epoch 4/500
11/11 [==============================] - 0s 698us/step - loss: 0.4942 - mean_squared_error: 0.4942
Epoch 5/500
11/11 [==============================] - 0s 698us/step - loss: 0.4385 - mean_squared_error: 0.4385
Epoch 6/500
11/11 [==============================] - 0s 700us/step - loss: 0.4160 - mean_squared_error: 0.4160
Epoch 7/500
11/11 [==============================] - 0s 898us/step - loss: 0.4055 - mean_squared_error: 0.4055
Epoch 8/500
11/11 [==============================] - 0s 798us/step - loss: 0.3947 - mean_squared_error: 0.3947
Ep

11/11 [==============================] - 0s 598us/step - loss: 0.2537 - mean_squared_error: 0.2537
Epoch 52/500
11/11 [==============================] - 0s 692us/step - loss: 0.2516 - mean_squared_error: 0.2516
Epoch 53/500
11/11 [==============================] - 0s 696us/step - loss: 0.2507 - mean_squared_error: 0.2507
Epoch 54/500
11/11 [==============================] - 0s 735us/step - loss: 0.2502 - mean_squared_error: 0.2502
Epoch 55/500
11/11 [==============================] - 0s 729us/step - loss: 0.2504 - mean_squared_error: 0.2504
Epoch 56/500
11/11 [==============================] - 0s 629us/step - loss: 0.2462 - mean_squared_error: 0.2462
Epoch 57/500
11/11 [==============================] - 0s 691us/step - loss: 0.2428 - mean_squared_error: 0.2428
Epoch 58/500
11/11 [==============================] - 0s 599us/step - loss: 0.2432 - mean_squared_error: 0.2432
Epoch 59/500
11/11 [==============================] - 0s 696us/step - loss: 0.2485 - mean_squared_error: 0.2485
Epoch

11/11 [==============================] - 0s 698us/step - loss: 0.1985 - mean_squared_error: 0.1985
Epoch 125/500
11/11 [==============================] - 0s 784us/step - loss: 0.1967 - mean_squared_error: 0.1967
Epoch 126/500
11/11 [==============================] - 0s 695us/step - loss: 0.2028 - mean_squared_error: 0.2028
Epoch 127/500
11/11 [==============================] - 0s 696us/step - loss: 0.1960 - mean_squared_error: 0.1960
Epoch 128/500
11/11 [==============================] - 0s 598us/step - loss: 0.1939 - mean_squared_error: 0.1939
Epoch 129/500
11/11 [==============================] - 0s 641us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 130/500
11/11 [==============================] - 0s 695us/step - loss: 0.1954 - mean_squared_error: 0.1954
Epoch 131/500
11/11 [==============================] - 0s 601us/step - loss: 0.1966 - mean_squared_error: 0.1966
Epoch 132/500
11/11 [==============================] - 0s 795us/step - loss: 0.1937 - mean_squared_error: 0.19

11/11 [==============================] - 0s 799us/step - loss: 0.1843 - mean_squared_error: 0.1843
Epoch 197/500
11/11 [==============================] - 0s 696us/step - loss: 0.1795 - mean_squared_error: 0.1795
Epoch 198/500
11/11 [==============================] - 0s 601us/step - loss: 0.1805 - mean_squared_error: 0.1805
Epoch 199/500
11/11 [==============================] - 0s 698us/step - loss: 0.1781 - mean_squared_error: 0.1781
Epoch 200/500
11/11 [==============================] - 0s 649us/step - loss: 0.1776 - mean_squared_error: 0.1776
Epoch 201/500
11/11 [==============================] - 0s 695us/step - loss: 0.1795 - mean_squared_error: 0.1795
Epoch 202/500
11/11 [==============================] - 0s 601us/step - loss: 0.1795 - mean_squared_error: 0.1795
Epoch 203/500
11/11 [==============================] - 0s 813us/step - loss: 0.1803 - mean_squared_error: 0.1803
Epoch 204/500
11/11 [==============================] - 0s 615us/step - loss: 0.1809 - mean_squared_error: 0.18

11/11 [==============================] - 0s 698us/step - loss: 0.1837 - mean_squared_error: 0.1837
Epoch 269/500
11/11 [==============================] - 0s 701us/step - loss: 0.1841 - mean_squared_error: 0.1841
Epoch 270/500
11/11 [==============================] - 0s 701us/step - loss: 0.1811 - mean_squared_error: 0.1811
Epoch 271/500
11/11 [==============================] - 0s 701us/step - loss: 0.1772 - mean_squared_error: 0.1772
Epoch 272/500
11/11 [==============================] - 0s 698us/step - loss: 0.1746 - mean_squared_error: 0.1746
Epoch 273/500
11/11 [==============================] - 0s 698us/step - loss: 0.1810 - mean_squared_error: 0.1810
Epoch 274/500
11/11 [==============================] - 0s 698us/step - loss: 0.1778 - mean_squared_error: 0.1778
Epoch 275/500
11/11 [==============================] - 0s 795us/step - loss: 0.1741 - mean_squared_error: 0.1741
Epoch 276/500
11/11 [==============================] - 0s 598us/step - loss: 0.1740 - mean_squared_error: 0.17

11/11 [==============================] - 0s 691us/step - loss: 0.1706 - mean_squared_error: 0.1706
Epoch 341/500
11/11 [==============================] - 0s 798us/step - loss: 0.1688 - mean_squared_error: 0.1688
Epoch 342/500
11/11 [==============================] - 0s 696us/step - loss: 0.1656 - mean_squared_error: 0.1656
Epoch 343/500
11/11 [==============================] - 0s 700us/step - loss: 0.1712 - mean_squared_error: 0.1712
Epoch 344/500
11/11 [==============================] - 0s 596us/step - loss: 0.1716 - mean_squared_error: 0.1716
Epoch 345/500
11/11 [==============================] - 0s 696us/step - loss: 0.1674 - mean_squared_error: 0.1674
Epoch 346/500
11/11 [==============================] - 0s 698us/step - loss: 0.1692 - mean_squared_error: 0.1692
Epoch 347/500
11/11 [==============================] - 0s 607us/step - loss: 0.1683 - mean_squared_error: 0.1683
Epoch 348/500
11/11 [==============================] - 0s 798us/step - loss: 0.1680 - mean_squared_error: 0.16

11/11 [==============================] - 0s 698us/step - loss: 0.1670 - mean_squared_error: 0.1670
Epoch 413/500
11/11 [==============================] - 0s 798us/step - loss: 0.1668 - mean_squared_error: 0.1668
Epoch 414/500
11/11 [==============================] - 0s 660us/step - loss: 0.1688 - mean_squared_error: 0.1688
Epoch 415/500
11/11 [==============================] - 0s 696us/step - loss: 0.1688 - mean_squared_error: 0.1688
Epoch 416/500
11/11 [==============================] - 0s 696us/step - loss: 0.1685 - mean_squared_error: 0.1685
Epoch 417/500
11/11 [==============================] - 0s 696us/step - loss: 0.1734 - mean_squared_error: 0.1734
Epoch 418/500
11/11 [==============================] - 0s 601us/step - loss: 0.1779 - mean_squared_error: 0.1779
Epoch 419/500
11/11 [==============================] - 0s 698us/step - loss: 0.1745 - mean_squared_error: 0.1745
Epoch 420/500
11/11 [==============================] - 0s 590us/step - loss: 0.1713 - mean_squared_error: 0.17

11/11 [==============================] - 0s 714us/step - loss: 0.1708 - mean_squared_error: 0.1708
Epoch 485/500
11/11 [==============================] - 0s 698us/step - loss: 0.1678 - mean_squared_error: 0.1678
Epoch 486/500
11/11 [==============================] - 0s 759us/step - loss: 0.1837 - mean_squared_error: 0.1837
Epoch 487/500
11/11 [==============================] - 0s 808us/step - loss: 0.1808 - mean_squared_error: 0.1808
Epoch 488/500
11/11 [==============================] - 0s 773us/step - loss: 0.1694 - mean_squared_error: 0.1694
Epoch 489/500
11/11 [==============================] - 0s 795us/step - loss: 0.1657 - mean_squared_error: 0.1657
Epoch 490/500
11/11 [==============================] - 0s 658us/step - loss: 0.1679 - mean_squared_error: 0.1679
Epoch 491/500
11/11 [==============================] - 0s 698us/step - loss: 0.1697 - mean_squared_error: 0.1697
Epoch 492/500
11/11 [==============================] - 0s 602us/step - loss: 0.1678 - mean_squared_error: 0.16

In [30]:
X_test_norm_dict = {name:np.array(value) for name, value in X_test_norm.items()}
y_test_norm_dict = {y_test_norm.name:y_test_norm.values}

In [31]:
model.evaluate(X_test_norm_dict, y_test_norm_dict.values())

Consider rewriting this model with the Functional API.
2/2 [==============================] - 0s 2ms/step - loss: 0.1072 - mean_squared_error: 0.1072


[0.10722622275352478, 0.10722622275352478]